In [1]:
%load_ext autoreload
%autoreload 2
from utils import dataloader, utils, calibrate_imu_stereo
import torch
from models.vgg_encoder_decoder import VGGNet, FCN8s
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
from scipy.spatial.transform import Rotation
import numpy as np
import math
from tqdm import tqdm
import gc

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### returns imu and stereo rotation matrices for a sequence

In [2]:
def rots_per_sequence(dataset, model, norm, calibration_mat):
    """
    Computes roll and pitch via stereo and aligns it to the imu. Performs it on the whole sequence.
    
    Parameters
    ----------
    dataset : torch dataset
        contains stereo images, calibration data and imu data
    model : nn.Model
        segmentation CNN (VGG based Encoder Decoder)
    norm : torch transformation
        function to normalize images for CNN input
    calibration_mat : np.array
        calibration matrix to align stereo and imu
    return: tuple
        tuple[0] : stereo estimated orientation as 3x3 matrix (already calibrated to imu)
        tuple[1] : imu orientation as 3x3 matrix
    """
    imu_roll_l = []
    imu_pitch_l = []
    stereo_roll_l = []
    stereo_pitch_l = []
    stereo_calibrated_mats = []
    stereo_uncalibrated_mats = []
    imu_mats = []

    for j in tqdm(range(len(dataset))):
        sample = dataset[j]
        
        img_l = sample["img_l"] / 255
        K_l = sample["K_l"]
        D_l = sample["D_l"]
        R_l = sample["R_l"]
        P_l = sample["P_l"]
        img_r = sample["img_r"] / 255
        K_r = sample["K_r"]
        D_r = sample["D_r"]
        R_r = sample["R_r"]
        P_r = sample["P_r"]
        R = sample["R"]
        T = sample["T"]
        imu = sample["imu"]
    
        # Intrisic rotation aerospace sequence (Z-Y’-X”) yaw, pitch and roll
        # quaternion to matrix conversion: https://www.euclideanspace.com/maths/geometry/rotations/conversions/quaternionToMatrix/index.htm
        imu = Rotation.from_quat(imu).as_euler("ZYX", degrees=False)
        imu_mat = Rotation.from_euler("ZYX", [0., imu[1], imu[2]], degrees=False).as_matrix()

        map1_left, map2_left = cv2.initUndistortRectifyMap(K_l, D_l, R_l, P_l, (img_l.shape[1], img_l.shape[0]), cv2.CV_32FC1)
        map1_right, map2_right = cv2.initUndistortRectifyMap(K_r, D_r, R_r, P_r, (img_r.shape[1], img_r.shape[0]), cv2.CV_32FC1)
        
        img_l = cv2.remap(img_l, map1_left, map2_left, cv2.INTER_LINEAR)
        img_r = cv2.remap(img_r, map1_right, map2_right, cv2.INTER_LINEAR)


        ################ estimate horizon with plane regression ####################################################################
        res_img = cv2.resize(img_l, dsize=(960, 640), interpolation=cv2.INTER_CUBIC)
        # Perform forward pass with resized image as input tensor
        trans1 = transforms.ToTensor()
        input_tensor = norm(trans1(res_img)).to("cuda:0")
        with torch.no_grad():
            output = model(input_tensor.unsqueeze(0).float())

        # Binarize output with a high threshold (less false positives (but also less true positives))
        output = utils.binarize(torch.squeeze(output.data.cpu()), activation="sigmoid", threshold=0.5).numpy()
        res_out = cv2.resize(output, dsize=(img_l.shape[1], img_l.shape[0]), interpolation=cv2.INTER_CUBIC)
        res_out *= 255
        res_out = res_out.astype(np.uint8)
        # # Mask original image
        img_l_masked = cv2.bitwise_and(img_l, img_l, mask = res_out)

        img_l_masked_resized, img_r_resized, P_l_resize, P_r_resize = utils.resize_img_and_projection(img_l_masked, img_r, P_l, P_r, 0.5)

        pc_water, disparity = utils.compute_pointcloud((img_l_masked_resized * 255).astype("uint8"), (img_r_resized * 255).astype("uint8"), 
                                                        P_l_resize, P_r_resize, minDisparity=0, numDisparities=160, blocksize=9)
        #######################################
        pc_water = pc_water[pc_water[:,2]<=40]

        normal, support = utils.calc_plane(pc_water, var_index=[0,2,1])
        normal = np.asarray(normal)
        support = np.asarray(support)
        normal = 1 / np.sqrt(np.sum(normal**2)) * normal
                
        roll, pitch = utils.normal_to_euler(normal, degrees=False)
        stereo_uncalibrated_mat = Rotation.from_euler("ZYX", [0., pitch, roll], degrees=False).as_matrix()
        stereo_calibrated_mat = np.dot(stereo_uncalibrated_mat, calibration_mat)
        stereo_uncalibrated_mats.append(stereo_uncalibrated_mat)
        stereo_calibrated_mats.append(stereo_calibrated_mat)
        imu_mats.append(imu_mat)
                
    return stereo_calibrated_mats, imu_mats

### average misorientation angle (https://jsdajournal.springeropen.com/articles/10.1186/s40488-015-0032-x)

In [3]:
def print_ama(imu_mats, stereo_mats):
    """
    Computes and prints the AMA and confidence intervals
    
    Parameters
    ----------
    imu_mats : list
        list of imu measurements per timestep (3x3 matrices) 
    stereo_mats : list
        list of stereo estimated orientation per timestep (3x3 matrices)
    """
    ### create dataset for bootstrapping
    diff_mats = []
    for i in range(len(imu_mats)):
        stereo_calibrated_mat = stereo_mats[i]
        imu_mat = imu_mats[i]
        diff_mats.append(np.dot(imu_mat, stereo_calibrated_mat.T))
    diff_mats = np.array(diff_mats)

    ### compute average misorientation angle
    sum_diff_mats = np.zeros((3,3))
    for i in range(diff_mats.shape[0]):
        sum_diff_mats += diff_mats[i]
    avg_diff_mat = sum_diff_mats / diff_mats.shape[0]
    U,S,W = np.linalg.svd(avg_diff_mat)
    M = np.dot(U,W)

    AMA = []
    for i in range(diff_mats.shape[0]):
        mis = np.arccos((np.trace(np.dot(diff_mats[i].T, M)) -1) / 2)
        AMA.append(mis)
    print("Misorientation angle in degree: ", np.mean(np.degrees(AMA)))


    ### Bootstrapping ###
    bootstrepping_iterations = 1000
    AMA_per_sample = []
    for k in range(bootstrepping_iterations):
        sample_idx = np.random.randint(0, diff_mats.shape[0], diff_mats.shape[0])
        sample = diff_mats[sample_idx]

        sum_diff_mats = np.zeros((3,3))
        for i in range(sample.shape[0]):
            sum_diff_mats += sample[i]
        avg_diff_mat = sum_diff_mats / sample.shape[0]
        U,S,W = np.linalg.svd(avg_diff_mat)
        M = np.dot(U,W)

        AMA = []
        for i in range(sample.shape[0]):
            mis = np.arccos((np.trace(np.dot(sample[i].T, M)) -1) / 2)
            AMA.append(mis)

        sum_ama = 0
        for i in range(len(AMA)):
            sum_ama += AMA[i]
        AMA_per_sample.append(np.degrees(sum_ama / len(AMA)))

    print("Confidence Intervalls in degrees (2.5, 97.5) quantile: ", np.quantile(AMA_per_sample, 0.025), np.quantile(AMA_per_sample, 0.975))

In [4]:
def get_gyro_measurments(dataset):
    """
    return gyro measurements and integrated orientations
    
    Parameters
    ----------
    dataset : torch dataset
        contains gyro measurements 
    
    return: tuple
        tuple[0] : integrated orientations 
        tuple[1] : gyro measurements
    """
    integrated_mats = []
    gyro_mats = [Rotation.from_quat(dataset[0]["dq"]).as_matrix()]
    integrated_measurements = Rotation.from_quat(dataset[0]["imu"]).as_matrix()
    integrated_mats.append(integrated_measurements)
    for i in range(1, len(dataset)): 
        gyro = dataset[i]["dq"]
        gyro = Rotation.from_quat(gyro).as_matrix()
        integrated_measurements = integrated_measurements @ gyro
        integrated_mats.append(integrated_measurements)
        gyro_mats.append(gyro)
    return integrated_mats, gyro_mats

In [5]:
def complementary_filter(stereo_mats, gyro_mats):
    """
    Fusion of stereo estimation and gyro data via complementary filter
    
    Parameters
    ----------
    stereo_mats : list
        contains stereo orientation measurements (3x3 matrices)
    gyro_mats : list
        contains gyro measurements (3x3 matrices)
    
    return: list
        fused orientation (3x3 matrices)
    """
    roll = 0
    pitch = 0
    filtered_mats = []
    for i in range(len(stereo_mats)):
        stereo_euler = Rotation.from_matrix(stereo_mats[i]).as_euler("ZYX", degrees=False)
        gyro_euler = Rotation.from_matrix(gyro_mats[i]).as_euler("ZYX", degrees=False)
        stereo_pitch = stereo_euler[1]
        stereo_roll = stereo_euler[2]
        gyro_pitch = gyro_euler[1]
        gyro_roll = gyro_euler[2]

        # filter
        roll = 0.8 * (roll + (gyro_roll * (1. / 1))) + 0.2 * stereo_roll        
        pitch = 0.8 * (pitch + (gyro_pitch * (1. / 1))) + 0.2 * stereo_pitch

        filtered_mats.append(Rotation.from_euler("ZYX", [0, pitch, roll]).as_matrix())
    return filtered_mats

### load model parameters

In [6]:
VGG_MODEL_PATH = "models/VGG16_Enc_Dec_Misc_Constance"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
vgg_model = VGGNet(requires_grad=True, remove_fc=True)
model = FCN8s(pretrained_net=vgg_model, n_class=1).to(device)
model.load_state_dict(torch.load(VGG_MODEL_PATH))
model.eval()
# Mean and standard deviation for all channels of the 'WaterMisc' Dataset
mean, std = ([0.4454203248023987, 0.4749860167503357, 0.4680652916431427], [0.2575828433036804, 0.2523757517337799, 0.2858140468597412])

# Transformation to normalize and unnormalize input images
norm = transforms.Normalize(mean, std)

#### Calibration Rotation for day_1

In [7]:
plt.rcParams['figure.figsize'] = [8, 8]
calibration_dataset = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=1, sequence="calib")
imu_stereo_calib = calibrate_imu_stereo.ImuStereoCalibration(model, norm, calibration_dataset, seq_from = 0, seq_to = len(calibration_dataset), degrees=True, debug=False, resize_factor=0.5)
calibration_mat_day_1 = imu_stereo_calib.calibrate(minDisparity = 0, numDisparities=160, blocksize=9)
del calibration_dataset

  0%|          | 0/596 [00:00<?, ?it/s]/home/dennis/programs/anaconda3/envs/ros/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 596/596 [07:05<00:00,  1.40it/s]


#### Evaluation day 1, sequence 0

In [8]:
dataset_d1_0 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=1, sequence="0")
stereo_mats_0, imu_mats_0 =  rots_per_sequence(dataset_d1_0, model, norm, calibration_mat_day_1)
integrated_gyro_mats_0, gyro_mats_0 = get_gyro_measurments(dataset_d1_0)
filtered_mats_0 = complementary_filter(stereo_mats_0, gyro_mats_0)
print_ama(imu_mats_0, stereo_mats_0)
print_ama(imu_mats_0, integrated_gyro_mats_0)
print_ama(imu_mats_0, filtered_mats_0)

100%|██████████| 753/753 [08:34<00:00,  1.46it/s]


Misorientation angle in degree:  0.1405267720257239
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.13572379799091022 0.14570942937096004
Misorientation angle in degree:  11.672481132405375
Confidence Intervalls in degrees (2.5, 97.5) quantile:  11.366254421465316 11.95163419191706
Misorientation angle in degree:  0.1300481411332243
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.12401130609019569 0.13707435897965245


In [9]:
del dataset_d1_0

#### Evaluation day 1, sequence 1

In [10]:
dataset_d1_1 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=1, sequence="1")
stereo_mats_1, imu_mats_1 =  rots_per_sequence(dataset_d1_1, model, norm, calibration_mat_day_1)
integrated_gyro_mats_1, gyro_mats_1 = get_gyro_measurments(dataset_d1_1)
filtered_mats_1 = complementary_filter(stereo_mats_1, gyro_mats_1)
print_ama(imu_mats_1, stereo_mats_1)
print_ama(imu_mats_1, integrated_gyro_mats_1)
print_ama(imu_mats_1, filtered_mats_1)

100%|██████████| 1119/1119 [12:58<00:00,  1.44it/s]


Misorientation angle in degree:  0.2038948554180303
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.1949877176653827 0.21449733778612695
Misorientation angle in degree:  26.347055061096075
Confidence Intervalls in degrees (2.5, 97.5) quantile:  24.866748925873164 27.863978060806993
Misorientation angle in degree:  0.1995186421052608
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.18795871759470026 0.21169565886264838


In [11]:
del dataset_d1_1

#### Evaluation day 1, sequence 2

In [12]:
dataset_d1_2 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=1, sequence="2")
stereo_mats_2, imu_mats_2 =  rots_per_sequence(dataset_d1_2, model, norm, calibration_mat_day_1)
integrated_gyro_mats_2, gyro_mats_2 = get_gyro_measurments(dataset_d1_2)
filtered_mats_2 = complementary_filter(stereo_mats_2, gyro_mats_2)
print_ama(imu_mats_2, stereo_mats_2)
print_ama(imu_mats_2, integrated_gyro_mats_2)
print_ama(imu_mats_2, filtered_mats_2)

100%|██████████| 826/826 [09:16<00:00,  1.48it/s]


Misorientation angle in degree:  0.16713998263932647
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.15999488467746234 0.17421170539080028
Misorientation angle in degree:  8.646174361317128
Confidence Intervalls in degrees (2.5, 97.5) quantile:  8.40364934973076 8.898534906764347
Misorientation angle in degree:  0.12960084563009666
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.12245350983251249 0.13766551045228503


In [13]:
del dataset_d1_2

#### Evaluation day 1, sequence 3

In [14]:
dataset_d1_3 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=1, sequence="3")
stereo_mats_3, imu_mats_3 =  rots_per_sequence(dataset_d1_3, model, norm, calibration_mat_day_1)
integrated_gyro_mats_3, gyro_mats_3 = get_gyro_measurments(dataset_d1_3)
filtered_mats_3 = complementary_filter(stereo_mats_3, gyro_mats_3)
print_ama(imu_mats_3, stereo_mats_3)
print_ama(imu_mats_3, integrated_gyro_mats_3)
print_ama(imu_mats_3, filtered_mats_3)

100%|██████████| 2390/2390 [28:06<00:00,  1.42it/s]


Misorientation angle in degree:  0.17651464467062228
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.17209247403070724 0.18081391624610849
Misorientation angle in degree:  23.86462608361848
Confidence Intervalls in degrees (2.5, 97.5) quantile:  23.357280203770312 24.361972648790115
Misorientation angle in degree:  0.12282306564668692
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.11657911733497651 0.1305717181232694


In [15]:
del dataset_d1_3

#### Evaluation day 1, sequence 4

In [16]:
dataset_d1_4 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=1, sequence="4")
stereo_mats_4, imu_mats_4 =  rots_per_sequence(dataset_d1_4, model, norm, calibration_mat_day_1)
integrated_gyro_mats_4, gyro_mats_4 = get_gyro_measurments(dataset_d1_4)
filtered_mats_4 = complementary_filter(stereo_mats_4, gyro_mats_4)
print_ama(imu_mats_4, stereo_mats_4)
print_ama(imu_mats_4, integrated_gyro_mats_4)
print_ama(imu_mats_4, filtered_mats_4)

100%|██████████| 597/597 [06:56<00:00,  1.43it/s]


Misorientation angle in degree:  0.15163037957490844
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.14425592849425542 0.15811573894852182
Misorientation angle in degree:  8.180498987118686
Confidence Intervalls in degrees (2.5, 97.5) quantile:  7.733577700793308 8.601247347136308
Misorientation angle in degree:  0.1358379146221798
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.12602035754918955 0.14736669266110777


In [17]:
del dataset_d1_4

#### Calibration Rotation for day_2

In [18]:
plt.rcParams['figure.figsize'] = [8, 8]
calibration_dataset = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=2, sequence="calib")
imu_stereo_calib = calibrate_imu_stereo.ImuStereoCalibration(model, norm, calibration_dataset, seq_from = 0, seq_to = len(calibration_dataset), degrees=True, debug=False, resize_factor=0.5)
calibration_mat_day_2 = imu_stereo_calib.calibrate(minDisparity = 0, numDisparities=160, blocksize=9)
del calibration_dataset

100%|██████████| 596/596 [06:55<00:00,  1.43it/s]


#### Evaluation day 2, sequence 5

In [19]:
dataset_d2_5 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=2, sequence="5")
stereo_mats_5, imu_mats_5 =  rots_per_sequence(dataset_d2_5, model, norm, calibration_mat_day_2)
integrated_gyro_mats_5, gyro_mats_5 = get_gyro_measurments(dataset_d2_5)
filtered_mats_5 = complementary_filter(stereo_mats_5, gyro_mats_5)
print_ama(imu_mats_5, stereo_mats_5)
print_ama(imu_mats_5, integrated_gyro_mats_5)
print_ama(imu_mats_5, filtered_mats_5)

100%|██████████| 597/597 [07:05<00:00,  1.40it/s]


Misorientation angle in degree:  0.18315330602343452
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.17320805543375203 0.19239929207480075
Misorientation angle in degree:  35.80244279848965
Confidence Intervalls in degrees (2.5, 97.5) quantile:  33.4758694935343 37.88542388152431
Misorientation angle in degree:  0.13510833272362988
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.12519283233793443 0.14654956268413305


In [20]:
del dataset_d2_5

#### Evaluation day 2, sequence 6

In [21]:
dataset_d2_6 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=2, sequence="6")
stereo_mats_6, imu_mats_6 =  rots_per_sequence(dataset_d2_6, model, norm, calibration_mat_day_2)
integrated_gyro_mats_6, gyro_mats_6 = get_gyro_measurments(dataset_d2_6)
filtered_mats_6 = complementary_filter(stereo_mats_6, gyro_mats_6)
print_ama(imu_mats_6, stereo_mats_6)
print_ama(imu_mats_6, integrated_gyro_mats_6)
print_ama(imu_mats_6, filtered_mats_6)

100%|██████████| 439/439 [05:22<00:00,  1.36it/s]


Misorientation angle in degree:  0.18331497454670886
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.1723665907833313 0.19229792359908196
Misorientation angle in degree:  5.7173529263820875
Confidence Intervalls in degrees (2.5, 97.5) quantile:  5.478523922058683 5.91700174553075
Misorientation angle in degree:  0.15920080835617495
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.1355993380643809 0.18643792102330617


In [22]:
del dataset_d2_6

#### Evaluation day 2, sequence 7

In [23]:
dataset_d2_7 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=2, sequence="7")
stereo_mats_7, imu_mats_7 =  rots_per_sequence(dataset_d2_7, model, norm, calibration_mat_day_2)
integrated_gyro_mats_7, gyro_mats_7 = get_gyro_measurments(dataset_d2_7)
filtered_mats_7 = complementary_filter(stereo_mats_7, gyro_mats_7)
print_ama(imu_mats_7, stereo_mats_7)
print_ama(imu_mats_7, integrated_gyro_mats_7)
print_ama(imu_mats_7, filtered_mats_7)

100%|██████████| 1016/1016 [13:36<00:00,  1.24it/s]


Misorientation angle in degree:  1.6040704358249624
Confidence Intervalls in degrees (2.5, 97.5) quantile:  1.4401604798374532 1.78634812391366
Misorientation angle in degree:  11.997248570189218
Confidence Intervalls in degrees (2.5, 97.5) quantile:  11.49057288300878 12.481259168253423
Misorientation angle in degree:  0.9674718511320815
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.9086228084068327 1.029257888807806


In [24]:
del dataset_d2_7

#### Evaluation day 2, sequence 8

In [25]:
dataset_d2_8 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=2, sequence="8")
stereo_mats_8, imu_mats_8 =  rots_per_sequence(dataset_d2_8, model, norm, calibration_mat_day_2)
integrated_gyro_mats_8, gyro_mats_8 = get_gyro_measurments(dataset_d2_8)
filtered_mats_8 = complementary_filter(stereo_mats_8, gyro_mats_8)
print_ama(imu_mats_8, stereo_mats_8)
print_ama(imu_mats_8, integrated_gyro_mats_8)
print_ama(imu_mats_8, filtered_mats_8)

100%|██████████| 584/584 [06:59<00:00,  1.39it/s]


Misorientation angle in degree:  0.42662841341943014
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.39991936408273737 0.4548578395931814
Misorientation angle in degree:  13.01855675551615
Confidence Intervalls in degrees (2.5, 97.5) quantile:  12.384965973043919 13.623500395942905
Misorientation angle in degree:  0.23249645310382994
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.21858862679326727 0.24702029056115415


In [26]:
del dataset_d2_8

In [27]:
imu_mats = (imu_mats_0 + imu_mats_1)
stereo_mats = (stereo_mats_0 + stereo_mats_1 + stereo_mats_2 + stereo_mats_3 + stereo_mats_4 + stereo_mats_5 + stereo_mats_6 + stereo_mats_7 + stereo_mats_8)
integrated_gyro_mats = (integrated_gyro_mats_0 + integrated_gyro_mats_1 + integrated_gyro_mats_2 + integrated_gyro_mats_3 + integrated_gyro_mats_4 + integrated_gyro_mats_5 + integrated_gyro_mats_6 + integrated_gyro_mats_7 + integrated_gyro_mats_8)
filtered_mats = (filtered_mats_0 + filtered_mats_1 + filtered_mats_2 + filtered_mats_3 + filtered_mats_4 + filtered_mats_5 + filtered_mats_6 + filtered_mats_7 + filtered_mats_8)

print_ama(imu_mats, stereo_mats)
print_ama(imu_mats, integrated_gyro_mats)
print_ama(imu_mats, filtered_mats)

Misorientation angle in degree:  0.18556148907795714
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.17890876016886462 0.19206838122756512
Misorientation angle in degree:  40.61837018363317
Confidence Intervalls in degrees (2.5, 97.5) quantile:  39.557591758582575 41.66153157901051
Misorientation angle in degree:  0.17874917365804138
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.1709215781148951 0.18633206775766925
